In [2]:
# Install required libraries
!pip install -q datasets sacrebleu rouge-score

from google.colab import drive
drive.mount("/content/drive")

import os
import json
import random
import math
import time
from collections import Counter

import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset
import sacrebleu
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt


def set_seed(seed: int = 42) -> None:
    """
    Set global random seeds and deterministic flags.

    - Key responsibilities: configure Python, NumPy, and PyTorch RNG; enforce deterministic cuDNN behavior when possible.
    - Return values described: returns None.
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Project paths on Drive


BASE_DIR = "/content/drive/MyDrive/TakeHome543/Question2"
DIRS = {
    "data": os.path.join(BASE_DIR, "data"),
    "vocab": os.path.join(BASE_DIR, "vocab"),
    "checkpoints": os.path.join(BASE_DIR, "checkpoints"),
    "logs": os.path.join(BASE_DIR, "logs"),
    "figs": os.path.join(BASE_DIR, "figs"),
}


def ensure_project_directories() -> None:
    """
    Create all required project directories on Drive.

    - Key responsibilities: create data, vocab, checkpoints, logs, and figs directories if they do not exist.
    - Return values described: returns None.
    """

    for p in DIRS.values():
        os.makedirs(p, exist_ok=True)


set_seed(42)
ensure_project_directories()
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)


# Hyperparameters

MIN_FREQ = 3
MAX_VOCAB_SIZE = 20000
MAX_LEN = 30

SPECIAL_TOKENS = {
    "<pad>": 0,
    "<bos>": 1,
    "<eos>": 2,
    "<unk>": 3,
}

PAD_IDX = SPECIAL_TOKENS["<pad>"]
BOS_IDX = SPECIAL_TOKENS["<bos>"]
EOS_IDX = SPECIAL_TOKENS["<eos>"]
UNK_IDX = SPECIAL_TOKENS["<unk>"]

BATCH_SIZE = 64
EMB_SIZE = 256
HIDDEN_SIZE = 256
ENC_LAYERS = 1
DEC_LAYERS = 1
DROPOUT = 0.2
LR = 1e-3
CLIP = 1.0
MAX_DECODE_LEN = MAX_LEN

# Attention-specific epoch configuration
EPOCHS_CONFIG = {
    "bahdanau": 10,
    "luong": 15,
    "scaled_dot": 12,
}
DEFAULT_EPOCHS = 5  # safety fallback


def simple_tokenize(text: str):
    """
    Perform minimal whitespace tokenization.

    - Key responsibilities: lowercase input text; split by whitespace; trim surrounding spaces.
    - Return values described: returns a list of token strings.
    """

    return text.strip().lower().split()



# STAGE 1: Dataset & Vocab

def load_quora_paraphrase_splits(seed: int = 42):
    """
    Load Quora paraphrase subset and create train/valid/test splits.

    - Key responsibilities: filter paraphrase pairs (is_duplicate == 1); shuffle and partition into train, validation, and test splits.
    - Return values described: returns (train_split, valid_split, test_split) as HuggingFace dataset objects.
    """

    raw = load_dataset("AlekseyKorshuk/quora-question-pairs")
    dup = raw["train"].filter(lambda ex: ex["is_duplicate"] == 1)
    dup = dup.shuffle(seed=seed)

    n_total = len(dup)
    n_train = int(0.8 * n_total)
    n_valid = int(0.1 * n_total)
    n_test = n_total - n_train - n_valid

    train_split = dup.select(range(n_train))
    valid_split = dup.select(range(n_train, n_train + n_valid))
    test_split = dup.select(range(n_train + n_valid, n_total))

    return train_split, valid_split, test_split


train_ds, valid_ds, test_ds = load_quora_paraphrase_splits(seed=42)
print("Quora paraphrase sizes:", len(train_ds), len(valid_ds), len(test_ds))


def build_vocab_from_train(train_split, min_freq: int, max_vocab_size: int):
    """
    Build source-target shared vocabulary from training split.

    - Key responsibilities: count token frequencies, select most frequent tokens, construct stoi and itos mappings with special tokens.
    - Return values described: returns (stoi, itos) where stoi maps token to index and itos maps index to token.
    """

    counter = Counter()
    for ex in train_split:
        counter.update(simple_tokenize(ex["question1"]))
        counter.update(simple_tokenize(ex["question2"]))

    filtered = [(w, c) for w, c in counter.items() if c >= min_freq]
    filtered.sort(key=lambda x: x[1], reverse=True)
    filtered = filtered[: (max_vocab_size - len(SPECIAL_TOKENS))]

    itos = [None] * len(SPECIAL_TOKENS) + [w for w, _ in filtered]
    stoi = {w: i for i, w in enumerate(itos)}

    for tok, idx in SPECIAL_TOKENS.items():
        itos[idx] = tok
        stoi[tok] = idx

    return stoi, itos


stoi, itos = build_vocab_from_train(train_ds, MIN_FREQ, MAX_VOCAB_SIZE)
VOCAB_SIZE = len(itos)
print("Vocab size:", VOCAB_SIZE)


def save_vocab(path: str, stoi_dict, itos_list) -> None:
    """
    Save vocabulary mappings to JSON on Drive.

    - Key responsibilities: serialize stoi and itos structures into a JSON file under the given path.
    - Return values described: returns None.
    """

    obj = {"stoi": stoi_dict, "itos": itos_list}
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, indent=2, ensure_ascii=False)


save_vocab(os.path.join(DIRS["vocab"], "vocab.json"), stoi, itos)
print("Saved vocab.json to:", os.path.join(DIRS["vocab"], "vocab.json"))


def encode(text: str, max_len: int = MAX_LEN):
    """
    Encode raw text into a fixed-length sequence of token IDs.

    - Key responsibilities: tokenize text, truncate to max_len-2, add BOS/EOS, and pad with PAD tokens to fixed length.
    - Return values described: returns a list of integer token IDs.
    """

    tokens = simple_tokenize(text)
    tokens = tokens[: (max_len - 2)]
    ids = [BOS_IDX]
    for t in tokens:
        ids.append(stoi.get(t, UNK_IDX))
    ids.append(EOS_IDX)
    if len(ids) < max_len:
        ids += [PAD_IDX] * (max_len - len(ids))
    return ids


def decode(ids):
    """
    Decode a sequence of token IDs back to tokens.

    - Key responsibilities: skip PAD tokens, stop at EOS, map indices to tokens via vocabulary.
    - Return values described: returns a list of token strings.
    """

    tokens = []
    for i in ids:
        if i == PAD_IDX:
            continue
        if i == EOS_IDX:
            break
        if 0 <= i < len(itos):
            tokens.append(itos[i])
        else:
            tokens.append("<unk>")
    return tokens


def save_tokenized_jsonl(split, path: str, max_items: int = 20000) -> None:
    """
    Save a tokenized snapshot of a dataset split to JSONL.

    - Key responsibilities: encode questions to ID sequences; serialize a subset to JSONL for inspection and reproducibility.
    - Return values described: returns None.
    """

    count = 0
    with open(path, "w", encoding="utf-8") as f:
        for ex in split:
            src_ids = encode(ex["question1"])
            tgt_ids = encode(ex["question2"])
            obj = {
                "question1": ex["question1"],
                "question2": ex["question2"],
                "src_ids": src_ids,
                "tgt_ids": tgt_ids,
            }
            f.write(json.dumps(obj) + "\n")
            count += 1
            if count >= max_items:
                break


save_tokenized_jsonl(train_ds, os.path.join(DIRS["data"], "train.jsonl"))
save_tokenized_jsonl(valid_ds, os.path.join(DIRS["data"], "valid.jsonl"))
save_tokenized_jsonl(test_ds, os.path.join(DIRS["data"], "test.jsonl"))
print("Saved tokenized JSONL snapshots under:", DIRS["data"])


# STAGE 2: Dataset & Loader

class QuoraParaphraseDataset(Dataset):
    """
    Quora paraphrase pairs as a seq2seq dataset.

    - Key responsibilities: encode question1 as source, encode question2 as target, prepare decoder inputs and outputs for teacher forcing.
    - Return values described: returns dictionary with tensors for src, dec_in, dec_out, and raw texts.
    """

    def __init__(self, hf_split):
        self.data = hf_split

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        ex = self.data[idx]
        src_ids = encode(ex["question1"])
        tgt_ids = encode(ex["question2"])

        dec_in = tgt_ids[:-1]
        dec_out = tgt_ids[1:]

        return {
            "src": torch.tensor(src_ids, dtype=torch.long),
            "dec_in": torch.tensor(dec_in, dtype=torch.long),
            "dec_out": torch.tensor(dec_out, dtype=torch.long),
            "src_text": ex["question1"],
            "tgt_text": ex["question2"],
        }


train_data = QuoraParaphraseDataset(train_ds)
valid_data = QuoraParaphraseDataset(valid_ds)
test_data = QuoraParaphraseDataset(test_ds)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

print("Batches:", len(train_loader), len(valid_loader), len(test_loader))



# STAGE 3: Attention Modules

class BahdanauAttention(nn.Module):
    """
    Implement additive (Bahdanau) attention.

    - Key responsibilities: compute alignment scores via learned projections, apply softmax with optional masking, form context vectors.
    - Return values described: returns (context, attention_weights) for each batch element.
    """

    def __init__(self, hidden_size: int):
        super().__init__()
        self.W_h = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_s = nn.Linear(hidden_size, hidden_size, bias=False)
        self.v = nn.Linear(hidden_size, 1, bias=False)

    def forward(self, decoder_hidden, encoder_outputs, mask=None):
        proj_enc = self.W_h(encoder_outputs)
        proj_dec = self.W_s(decoder_hidden).unsqueeze(1)
        scores = self.v(torch.tanh(proj_enc + proj_dec)).squeeze(-1)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn_weights = torch.softmax(scores, dim=-1)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attn_weights


class LuongAttention(nn.Module):
    """
    Implement multiplicative (Luong) attention.

    - Key responsibilities: project decoder hidden state, compute dot-product scores, apply masking and softmax, form context vectors.
    - Return values described: returns (context, attention_weights) for each batch element.
    """

    def __init__(self, hidden_size: int):
        super().__init__()
        self.W = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, decoder_hidden, encoder_outputs, mask=None):
        proj_dec = self.W(decoder_hidden).unsqueeze(2)
        scores = torch.bmm(encoder_outputs, proj_dec).squeeze(-1)

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn_weights = torch.softmax(scores, dim=-1)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attn_weights


class ScaledDotAttention(nn.Module):
    """
    Implement single-head scaled dot-product attention.

    - Key responsibilities: compute scaled dot-product scores between decoder hidden and encoder outputs, apply masking and softmax, form context vectors.
    - Return values described: returns (context, attention_weights) for each batch element.
    """

    def __init__(self, hidden_size: int):
        super().__init__()
        self.scale = math.sqrt(hidden_size)

    def forward(self, decoder_hidden, encoder_outputs, mask=None):
        scores = torch.bmm(encoder_outputs, decoder_hidden.unsqueeze(2)).squeeze(-1)
        scores = scores / self.scale

        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)

        attn_weights = torch.softmax(scores, dim=-1)
        context = torch.bmm(attn_weights.unsqueeze(1), encoder_outputs).squeeze(1)
        return context, attn_weights


# STAGE 4: Encoder–Decoder

class Encoder(nn.Module):
    """
    BiLSTM encoder for sequence-to-sequence modeling.

    - Key responsibilities: embed source token IDs, run bidirectional LSTM, project outputs and final states into decoder hidden space.
    - Return values described: returns (encoder_outputs, initial_decoder_state, source_mask).
    """

    def __init__(self, vocab_size, emb_size, hidden_size, num_layers, dropout):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.lstm = nn.LSTM(
            emb_size,
            hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
            bidirectional=True,
        )
        self.fc = nn.Linear(hidden_size * 2, hidden_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, src_ids):
        mask = (src_ids != PAD_IDX).int()
        emb = self.dropout(self.embedding(src_ids))
        outputs, (h_n, c_n) = self.lstm(emb)

        outputs = torch.tanh(self.fc(outputs))

        h_cat = torch.cat((h_n[-2], h_n[-1]), dim=-1)
        c_cat = torch.cat((c_n[-2], c_n[-1]), dim=-1)
        h0 = torch.tanh(self.fc(h_cat)).unsqueeze(0)
        c0 = torch.tanh(self.fc(c_cat)).unsqueeze(0)

        return outputs, (h0, c0), mask


class Decoder(nn.Module):
    """
    LSTM decoder with pluggable attention mechanism.

    - Key responsibilities: embed target inputs, compute attention-based context at each step, update decoder state, produce token logits.
    - Return values described: returns (logits_tensor, attention_tensor) over all time steps.
    """

    def __init__(self, vocab_size, emb_size, hidden_size, num_layers,
                 dropout, attention_type="bahdanau"):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=PAD_IDX)
        self.lstm = nn.LSTM(
            emb_size + hidden_size,
            hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            batch_first=True,
        )
        self.fc_out = nn.Linear(hidden_size, vocab_size)
        self.dropout = nn.Dropout(dropout)

        if attention_type == "bahdanau":
            self.attn = BahdanauAttention(hidden_size)
        elif attention_type == "luong":
            self.attn = LuongAttention(hidden_size)
        elif attention_type == "scaled_dot":
            self.attn = ScaledDotAttention(hidden_size)
        else:
            raise ValueError(f"Unknown attention type: {attention_type}")

        self.attention_type = attention_type

    def forward(self, dec_in_ids, encoder_outputs, encoder_hidden, src_mask):
        emb = self.dropout(self.embedding(dec_in_ids))
        batch_size, tgt_len, _ = emb.size()
        h, c = encoder_hidden
        h = h.contiguous()
        c = c.contiguous()

        logits_list = []
        attn_list = []

        for t in range(tgt_len):
            emb_t = emb[:, t, :]
            context, attn_weights = self.attn(h[-1], encoder_outputs, src_mask)
            lstm_input = torch.cat([emb_t, context], dim=-1).unsqueeze(1)
            output, (h, c) = self.lstm(lstm_input, (h, c))
            logit_t = self.fc_out(output.squeeze(1))

            logits_list.append(logit_t)
            attn_list.append(attn_weights)

        logits = torch.stack(logits_list, dim=1)
        attentions = torch.stack(attn_list, dim=1)
        return logits, attentions


class Seq2Seq(nn.Module):
    """
    Full encoder–decoder architecture with attention.

    - Key responsibilities: encode source sequences, decode target sequences with the chosen attention formulation, expose unified forward interface.
    - Return values described: returns (logits_tensor, attention_tensor) for the decoder outputs.
    """

    def __init__(self, vocab_size, emb_size, hidden_size,
                 enc_layers, dec_layers, dropout, attention_type="bahdanau"):
        super().__init__()
        self.encoder = Encoder(vocab_size, emb_size, hidden_size, enc_layers, dropout)
        self.decoder = Decoder(vocab_size, emb_size, hidden_size,
                               dec_layers, dropout, attention_type)

    def forward(self, src_ids, dec_in_ids):
        enc_out, enc_hidden, src_mask = self.encoder(src_ids)
        logits, attn = self.decoder(dec_in_ids, enc_out, enc_hidden, src_mask)
        return logits, attn


# STAGE 5: Training & Eval


def train_one_epoch(model, loader, optimizer, criterion):
    """
    Train the model for a single epoch.

    - Key responsibilities: iterate over training batches, compute cross-entropy loss, apply gradient clipping, update model parameters.
    - Return values described: returns the average training loss over all batches.
    """

    model.train()
    total_loss = 0.0

    for batch in loader:
        src = batch["src"].to(DEVICE)
        dec_in = batch["dec_in"].to(DEVICE)
        dec_out = batch["dec_out"].to(DEVICE)

        optimizer.zero_grad()
        logits, _ = model(src, dec_in)
        B, T, V = logits.size()
        loss = criterion(logits.view(B * T, V), dec_out.view(B * T))
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), CLIP)
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(loader)


def eval_loss(model, loader, criterion):
    """
    Evaluate cross-entropy loss on a data split.

    - Key responsibilities: disable gradients, compute average cross-entropy loss across all batches of the provided loader.
    - Return values described: returns the average evaluation loss.
    """

    model.eval()
    total_loss = 0.0

    with torch.no_grad():
        for batch in loader:
            src = batch["src"].to(DEVICE)
            dec_in = batch["dec_in"].to(DEVICE)
            dec_out = batch["dec_out"].to(DEVICE)

            logits, _ = model(src, dec_in)
            B, T, V = logits.size()
            loss = criterion(logits.view(B * T, V), dec_out.view(B * T))
            total_loss += loss.item()

    return total_loss / len(loader)


def loss_to_ppl(loss: float) -> float:
    """
    Convert average cross-entropy loss to perplexity.

    - Key responsibilities: apply exponential mapping from loss to perplexity for interpretability of language modeling quality.
    - Return values described: returns perplexity as a floating-point value.
    """

    return math.exp(loss)


def greedy_generate_batch(model, src_ids_batch, max_len: int = MAX_LEN):
    """
    Perform greedy decoding for a batch of source sequences.

    - Key responsibilities: repeatedly call the model with growing decoder input, select argmax tokens, and accumulate generated IDs.
    - Return values described: returns a list of generated ID lists, one per source example.
    """

    model.eval()
    src = src_ids_batch.to(DEVICE)
    B = src.size(0)

    dec_in = torch.full((B, 1), BOS_IDX, dtype=torch.long, device=DEVICE)
    generated = [[] for _ in range(B)]

    with torch.no_grad():
        for _ in range(max_len - 1):
            logits, _ = model(src, dec_in)
            next_ids = logits[:, -1, :].argmax(dim=-1)

            for i in range(B):
                generated[i].append(next_ids[i].item())

            dec_in = torch.cat([dec_in, next_ids.unsqueeze(1)], dim=1)

    return generated


def evaluate_text_metrics(model, data_loader, max_batches: int = 50):
    """
    Compute BLEU, ROUGE-L, and perplexity for the model.

    - Key responsibilities: run greedy decoding on a subset of batches, collect reference and hypothesis strings, compute BLEU, ROUGE-L, and test perplexity.
    - Return values described: returns a dictionary with keys 'bleu', 'rougeL', and 'ppl'.
    """

    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)
    model.eval()

    all_refs = []
    all_hyps = []
    total_loss = 0.0
    batches = 0

    with torch.no_grad():
        for batch in data_loader:
            src = batch["src"].to(DEVICE)
            dec_in = batch["dec_in"].to(DEVICE)
            dec_out = batch["dec_out"].to(DEVICE)

            logits, _ = model(src, dec_in)
            B, T, V = logits.size()
            loss = criterion(logits.view(B * T, V), dec_out.view(B * T))
            total_loss += loss.item()
            batches += 1

            gen_ids = greedy_generate_batch(model, src, max_len=MAX_LEN)

            for i in range(B):
                ref_tokens = decode(dec_out[i].cpu().tolist())
                hyp_tokens = decode(gen_ids[i])

                if len(ref_tokens) == 0 or len(hyp_tokens) == 0:
                    continue

                all_refs.append([" ".join(ref_tokens)])
                all_hyps.append(" ".join(hyp_tokens))

            if batches >= max_batches:
                break

    bleu = sacrebleu.corpus_bleu(all_hyps, list(zip(*all_refs)))
    scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
    rouge_scores = [
        scorer.score(r[0], h)["rougeL"].fmeasure
        for h, r in zip(all_hyps, all_refs)
    ]
    rouge_l = float(np.mean(rouge_scores))
    avg_loss = total_loss / batches
    ppl = loss_to_ppl(avg_loss)

    return {"bleu": float(bleu.score), "rougeL": rouge_l, "ppl": float(ppl)}


def attention_entropy(attn_weights: torch.Tensor) -> float:
    """
    Compute mean entropy of attention distributions.

    - Key responsibilities: compute token-level entropies over source positions and average across all time steps and batch elements.
    - Return values described: returns mean attention entropy as a float.
    """

    eps = 1e-8
    p = attn_weights.clamp(min=eps, max=1.0)
    ent = - (p * p.log()).sum(dim=-1)
    return float(ent.mean().item())


def collect_attention_stats(model, data_loader, max_batches: int = 50):
    """
    Collect summary statistics of attention entropy.

    - Key responsibilities: run the model on a subset of validation batches, compute entropies per batch, summarize mean and standard deviation.
    - Return values described: returns a dictionary with keys 'mean_entropy' and 'std_entropy'.
    """

    model.eval()
    entropies = []
    batches = 0

    with torch.no_grad():
        for batch in data_loader:
            src = batch["src"].to(DEVICE)
            dec_in = batch["dec_in"].to(DEVICE)
            logits, attn = model(src, dec_in)
            ent = attention_entropy(attn)
            entropies.append(ent)
            batches += 1
            if batches >= max_batches:
                break

    arr = np.array(entropies)
    return {"mean_entropy": float(arr.mean()), "std_entropy": float(arr.std())}


def visualize_attention_for_examples(model, data_loader, attention_type: str, num_examples: int = 5):
    """
    Visualize attention heatmaps for sample examples.

    - Key responsibilities: run the model on a few test examples, extract attention matrices, align them with decoded tokens, and save heatmaps to Drive.
    - Return values described: returns None.
    """

    model.eval()
    examples_done = 0
    meta_records = []

    for batch in data_loader:
        if examples_done >= num_examples:
            break

        src = batch["src"].to(DEVICE)
        dec_in = batch["dec_in"].to(DEVICE)
        dec_out = batch["dec_out"].to(DEVICE)
        src_texts = batch["src_text"]
        tgt_texts = batch["tgt_text"]

        with torch.no_grad():
            logits, attn = model(src, dec_in)

        B = src.size(0)
        for i in range(B):
            if examples_done >= num_examples:
                break

            src_tokens = decode(src[i].cpu().tolist())
            tgt_tokens = decode(dec_out[i].cpu().tolist())
            src_len = len(src_tokens)
            tgt_len = len(tgt_tokens)

            attn_matrix = attn[i, :tgt_len, :src_len].cpu().numpy()

            fig_w = max(4, src_len * 0.5)
            fig_h = max(4, tgt_len * 0.5)
            fig, ax = plt.subplots(figsize=(fig_w, fig_h))

            cax = ax.imshow(attn_matrix, aspect="auto",
                            interpolation="nearest", cmap="gray")
            fig.colorbar(cax)

            ax.set_xticks(range(src_len))
            ax.set_yticks(range(tgt_len))
            ax.set_xticklabels(src_tokens, rotation=90, fontsize=9)
            ax.set_yticklabels(tgt_tokens, fontsize=9)
            ax.set_xlabel("Source tokens", fontsize=11)
            ax.set_ylabel("Target tokens", fontsize=11)
            plt.title(f"Attention heatmap ({attention_type})")
            plt.tight_layout()

            fig_path = os.path.join(
                DIRS["figs"],
                f"attention_{attention_type}_{examples_done}.png"
            )
            plt.savefig(fig_path, dpi=200)
            plt.close(fig)

            meta_records.append(
                {
                    "index": examples_done,
                    "attention_type": attention_type,
                    "src_text": src_texts[i],
                    "tgt_text": tgt_texts[i],
                    "src_tokens": src_tokens,
                    "tgt_tokens": tgt_tokens,
                    "figure_path": fig_path,
                }
            )

            print(f"[{attention_type}] Example {examples_done}")
            print("  SRC:", src_texts[i])
            print("  TGT:", tgt_texts[i])
            print("  FIG:", fig_path)
            print("-" * 60)

            examples_done += 1

    meta_path = os.path.join(DIRS["logs"], f"attention_meta_{attention_type}.json")
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta_records, f, indent=2, ensure_ascii=False)
    print(f"Saved attention metadata to: {meta_path}")


def train_with_attention(attention_type: str):
    """
    Train the seq2seq model with a given attention formulation.

    - Key responsibilities: construct a new model instance, run multi-epoch training with attention-specific epochs, evaluate metrics, save checkpoints, logs, and attention visualizations.
    - Return values described: returns a tuple (model, history_list, metrics_dict, attention_stats_dict).
    """

    local_epochs = EPOCHS_CONFIG.get(attention_type, DEFAULT_EPOCHS)

    set_seed(42)
    model = Seq2Seq(
        vocab_size=VOCAB_SIZE,
        emb_size=EMB_SIZE,
        hidden_size=HIDDEN_SIZE,
        enc_layers=ENC_LAYERS,
        dec_layers=DEC_LAYERS,
        dropout=DROPOUT,
        attention_type=attention_type,
    ).to(DEVICE)

    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

    history = []
    for epoch in range(1, local_epochs + 1):
        start = time.time()
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        valid_loss = eval_loss(model, valid_loader, criterion)
        end = time.time()

        train_ppl = loss_to_ppl(train_loss)
        valid_ppl = loss_to_ppl(valid_loss)

        record = {
            "epoch": epoch,
            "train_loss": float(train_loss),
            "valid_loss": float(valid_loss),
            "train_ppl": float(train_ppl),
            "valid_ppl": float(valid_ppl),
            "time_sec": float(end - start),
        }
        history.append(record)

        print(
            f"[{attention_type}] Epoch {epoch:02d}/{local_epochs:02d} | "
            f"Train {train_loss:.3f} (PPL {train_ppl:.1f}) | "
            f"Valid {valid_loss:.3f} (PPL {valid_ppl:.1f}) | "
            f"Time {end - start:.1f}s"
        )

    ckpt_path = os.path.join(DIRS["checkpoints"], f"{attention_type}_final.pt")
    torch.save(model.state_dict(), ckpt_path)
    print(f"Saved checkpoint: {ckpt_path}")

    hist_path = os.path.join(DIRS["logs"], f"history_{attention_type}.json")
    with open(hist_path, "w", encoding="utf-8") as f:
        json.dump(history, f, indent=2)
    print(f"Saved training history: {hist_path}")

    metrics = evaluate_text_metrics(model, test_loader, max_batches=50)
    metrics_path = os.path.join(DIRS["logs"], f"metrics_{attention_type}.json")
    with open(metrics_path, "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2)
    print(f"Saved metrics: {metrics_path}")
    print(f"[{attention_type}] BLEU={metrics['bleu']:.2f}, ROUGE-L={metrics['rougeL']:.3f}, PPL={metrics['ppl']:.1f}")

    attn_stats = collect_attention_stats(model, valid_loader, max_batches=50)
    stats_path = os.path.join(DIRS["logs"], f"attention_stats_{attention_type}.json")
    with open(stats_path, "w", encoding="utf-8") as f:
        json.dump(attn_stats, f, indent=2)
    print(f"Saved attention stats: {stats_path}")
    print(f"[{attention_type}] mean entropy={attn_stats['mean_entropy']:.3f}, std={attn_stats['std_entropy']:.3f}")

    visualize_attention_for_examples(model, test_loader, attention_type, num_examples=5)

    return model, history, metrics, attn_stats



# MAIN: Train all 3 attentions


bahdanau_model, bahdanau_hist, bahdanau_metrics, bahdanau_attn_stats = train_with_attention("bahdanau")
luong_model, luong_hist, luong_metrics, luong_attn_stats = train_with_attention("luong")
scaled_model, scaled_hist, scaled_metrics, scaled_attn_stats = train_with_attention("scaled_dot")

print("All three attention variants trained and logged.")
print("Drive base directory:", BASE_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Device: cuda
Quora paraphrase sizes: 119410 14926 14927
Vocab size: 20000
Saved vocab.json to: /content/drive/MyDrive/TakeHome543/Question2/vocab/vocab.json
Saved tokenized JSONL snapshots under: /content/drive/MyDrive/TakeHome543/Question2/data
Batches: 1866 234 234


/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[bahdanau] Epoch 01/10 | Train 4.032 (PPL 56.4) | Valid 3.049 (PPL 21.1) | Time 156.1s
[bahdanau] Epoch 02/10 | Train 2.778 (PPL 16.1) | Valid 2.528 (PPL 12.5) | Time 155.9s
[bahdanau] Epoch 03/10 | Train 2.299 (PPL 10.0) | Valid 2.293 (PPL 9.9) | Time 154.9s
[bahdanau] Epoch 04/10 | Train 2.013 (PPL 7.5) | Valid 2.167 (PPL 8.7) | Time 155.5s
[bahdanau] Epoch 05/10 | Train 1.817 (PPL 6.2) | Valid 2.094 (PPL 8.1) | Time 155.0s
[bahdanau] Epoch 06/10 | Train 1.672 (PPL 5.3) | Valid 2.048 (PPL 7.7) | Time 155.2s
[bahdanau] Epoch 07/10 | Train 1.559 (PPL 4.8) | Valid 2.014 (PPL 7.5) | Time 154.6s
[bahdanau] Epoch 08/10 | Train 1.469 (PPL 4.3) | Valid 1.993 (PPL 7.3) | Time 155.2s
[bahdanau] Epoch 09/10 | Train 1.395 (PPL 4.0) | Valid 1.986 (PPL 7.3) | Time 155.1s
[bahdanau] Epoch 10/10 | Train 1.333 (PPL 3.8) | Valid 1.974 (PPL 7.2) | Time 155.3s
Saved checkpoint: /content/drive/MyDrive/TakeHome543/Question2/checkpoints/bahdanau_final.pt
Saved training history: /content/drive/MyDrive/TakeH

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[luong] Epoch 01/15 | Train 4.139 (PPL 62.7) | Valid 3.197 (PPL 24.5) | Time 134.8s
[luong] Epoch 02/15 | Train 2.937 (PPL 18.9) | Valid 2.682 (PPL 14.6) | Time 135.2s
[luong] Epoch 03/15 | Train 2.463 (PPL 11.7) | Valid 2.443 (PPL 11.5) | Time 135.2s
[luong] Epoch 04/15 | Train 2.174 (PPL 8.8) | Valid 2.312 (PPL 10.1) | Time 134.0s
[luong] Epoch 05/15 | Train 1.974 (PPL 7.2) | Valid 2.230 (PPL 9.3) | Time 134.2s
[luong] Epoch 06/15 | Train 1.824 (PPL 6.2) | Valid 2.183 (PPL 8.9) | Time 133.7s
[luong] Epoch 07/15 | Train 1.708 (PPL 5.5) | Valid 2.147 (PPL 8.6) | Time 134.1s
[luong] Epoch 08/15 | Train 1.613 (PPL 5.0) | Valid 2.131 (PPL 8.4) | Time 135.1s
[luong] Epoch 09/15 | Train 1.535 (PPL 4.6) | Valid 2.112 (PPL 8.3) | Time 135.0s
[luong] Epoch 10/15 | Train 1.466 (PPL 4.3) | Valid 2.103 (PPL 8.2) | Time 134.5s
[luong] Epoch 11/15 | Train 1.408 (PPL 4.1) | Valid 2.097 (PPL 8.1) | Time 135.9s
[luong] Epoch 12/15 | Train 1.357 (PPL 3.9) | Valid 2.100 (PPL 8.2) | Time 133.3s
[luong] E

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


[scaled_dot] Epoch 01/12 | Train 4.076 (PPL 58.9) | Valid 3.111 (PPL 22.4) | Time 128.0s
[scaled_dot] Epoch 02/12 | Train 2.839 (PPL 17.1) | Valid 2.579 (PPL 13.2) | Time 128.8s
[scaled_dot] Epoch 03/12 | Train 2.353 (PPL 10.5) | Valid 2.333 (PPL 10.3) | Time 128.9s
[scaled_dot] Epoch 04/12 | Train 2.060 (PPL 7.8) | Valid 2.205 (PPL 9.1) | Time 130.2s
[scaled_dot] Epoch 05/12 | Train 1.861 (PPL 6.4) | Valid 2.128 (PPL 8.4) | Time 128.6s
[scaled_dot] Epoch 06/12 | Train 1.712 (PPL 5.5) | Valid 2.075 (PPL 8.0) | Time 127.8s
[scaled_dot] Epoch 07/12 | Train 1.598 (PPL 4.9) | Valid 2.045 (PPL 7.7) | Time 127.2s
[scaled_dot] Epoch 08/12 | Train 1.504 (PPL 4.5) | Valid 2.021 (PPL 7.5) | Time 127.6s
[scaled_dot] Epoch 09/12 | Train 1.428 (PPL 4.2) | Valid 2.008 (PPL 7.4) | Time 127.3s
[scaled_dot] Epoch 10/12 | Train 1.363 (PPL 3.9) | Valid 2.001 (PPL 7.4) | Time 127.7s
[scaled_dot] Epoch 11/12 | Train 1.308 (PPL 3.7) | Valid 1.998 (PPL 7.4) | Time 127.4s
[scaled_dot] Epoch 12/12 | Train 1.26

In [3]:
import os
import json
import numpy as np

# Aynı base directory'yi kullanıyoruz
BASE_DIR = "/content/drive/MyDrive/TakeHome543/Question2"
LOG_DIR = os.path.join(BASE_DIR, "logs")
os.makedirs(LOG_DIR, exist_ok=True)

ATTN_TYPES = ["bahdanau", "luong", "scaled_dot"]


def load_json(path: str):
    """
    Load a JSON file from disk.

    - Key responsibilities: open the specified path, parse JSON content, handle missing files with a clear error.
    - Return values described: returns the deserialized Python object.
    """

    if not os.path.exists(path):
        raise FileNotFoundError(f"JSON file not found: {path}")
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def summarize_histories():
    """
    Summarize training histories for all attention types.

    - Key responsibilities: read history JSON files, extract final epoch validation loss and perplexity for each attention version.
    - Return values described: returns a dict mapping attention_type to {'final_valid_loss', 'final_valid_ppl'}.
    """

    summary = {}
    for attn in ATTN_TYPES:
        hist_path = os.path.join(LOG_DIR, f"history_{attn}.json")
        history = load_json(hist_path)
        last = history[-1]
        summary[attn] = {
            "final_valid_loss": float(last["valid_loss"]),
            "final_valid_ppl": float(last["valid_ppl"]),
        }
    return summary


def summarize_metrics_and_entropy():
    """
    Collect BLEU/ROUGE/PPL and attention entropy for all attention types.

    - Key responsibilities: read metrics_*.json and attention_stats_*.json files, aggregate scores for each attention mechanism.
    - Return values described: returns two dicts: metrics_dict and entropy_dict keyed by attention_type.
    """

    metrics = {}
    entropies = {}
    for attn in ATTN_TYPES:
        m_path = os.path.join(LOG_DIR, f"metrics_{attn}.json")
        s_path = os.path.join(LOG_DIR, f"attention_stats_{attn}.json")
        metrics[attn] = load_json(m_path)
        entropies[attn] = load_json(s_path)
    return metrics, entropies


def pretty_name(attn_type: str) -> str:
    """
    Map internal attention identifiers to human-readable labels.

    - Key responsibilities: convert 'bahdanau', 'luong', 'scaled_dot' into descriptive names for tables and figures.
    - Return values described: returns a formatted string label.
    """

    if attn_type == "bahdanau":
        return "Additive (Bahdanau)"
    if attn_type == "luong":
        return "Multiplicative (Luong)"
    if attn_type == "scaled_dot":
        return "Scaled dot-product"
    return attn_type


def build_latex_table(metrics, entropies, histories) -> str:
    """
    Build a LaTeX table summarizing attention variants.

    - Key responsibilities: assemble BLEU, ROUGE-L, PPL, mean entropy, and final validation perplexity into a LaTeX tabular environment.
    - Return values described: returns a LaTeX-formatted string ready to paste into a report.
    """

    rows = []
    for attn in ATTN_TYPES:
        m = metrics[attn]
        e = entropies[attn]
        h = histories[attn]
        row = (
            f"{pretty_name(attn)} & "
            f"{m['bleu']:.2f} & "
            f"{m['rougeL']:.3f} & "
            f"{m['ppl']:.1f} & "
            f"{h['final_valid_ppl']:.1f} & "
            f"{e['mean_entropy']:.3f} \\\\"
        )
        rows.append(row)

    rows_str = "\n".join(rows)

    latex = r"""
\begin{table}[h]
\centering
\begin{tabular}{lccccc}
\toprule
Attention type & BLEU $\uparrow$ & ROUGE-L $\uparrow$ & Test PPL $\downarrow$ & Val PPL (final) $\downarrow$ & Mean entropy $\downarrow$ \\
\midrule
""" + rows_str + r"""
\bottomrule
\end{tabular}
\caption{Comparison of three attention mechanisms on the Quora paraphrase generation task.}
\label{tab:attention-comparison}
\end{table}
"""
    return latex


def main_results_summary():
    """
    Aggregate experiment results and print a compact summary.

    - Key responsibilities: read all log JSON files, print numeric comparison, write a global summary JSON, and generate LaTeX table text.
    - Return values described: returns a dict with metrics, entropies, histories, and LaTeX table string.
    """

    histories = summarize_histories()
    metrics, entropies = summarize_metrics_and_entropy()

    print("=== Numerical summary per attention type ===")
    for attn in ATTN_TYPES:
        name = pretty_name(attn)
        m = metrics[attn]
        e = entropies[attn]
        h = histories[attn]
        print(f"\n{name}")
        print("-" * len(name))
        print(f"  BLEU           : {m['bleu']:.2f}")
        print(f"  ROUGE-L (F1)   : {m['rougeL']:.3f}")
        print(f"  Test PPL       : {m['ppl']:.1f}")
        print(f"  Final Val PPL  : {h['final_valid_ppl']:.1f}")
        print(f"  Mean entropy   : {e['mean_entropy']:.3f}")
        print(f"  Std entropy    : {e['std_entropy']:.3f}")

    latex_table = build_latex_table(metrics, entropies, histories)
    print("\n\n=== LaTeX table (copy-paste into report) ===\n")
    print(latex_table)

    summary_obj = {
        "histories": histories,
        "metrics": metrics,
        "entropies": entropies,
        "latex_table": latex_table,
    }
    out_path = os.path.join(LOG_DIR, "summary_all_attentions.json")
    with open(out_path, "w", encoding="utf-8") as f:
        json.dump(summary_obj, f, indent=2, ensure_ascii=False)
    print(f"\nSummary JSON written to: {out_path}")

    return summary_obj


# Run the aggregation
results_summary = main_results_summary()


=== Numerical summary per attention type ===

Additive (Bahdanau)
-------------------
  BLEU           : 26.30
  ROUGE-L (F1)   : 0.539
  Test PPL       : 7.3
  Final Val PPL  : 7.2
  Mean entropy   : 1.942
  Std entropy    : 0.030

Multiplicative (Luong)
----------------------
  BLEU           : 25.24
  ROUGE-L (F1)   : 0.514
  Test PPL       : 8.3
  Final Val PPL  : 8.2
  Mean entropy   : 0.000
  Std entropy    : 0.000

Scaled dot-product
------------------
  BLEU           : 26.10
  ROUGE-L (F1)   : 0.535
  Test PPL       : 7.5
  Final Val PPL  : 7.4
  Mean entropy   : 1.973
  Std entropy    : 0.033


=== LaTeX table (copy-paste into report) ===


\begin{table}[h]
\centering
\begin{tabular}{lccccc}
\toprule
Attention type & BLEU $\uparrow$ & ROUGE-L $\uparrow$ & Test PPL $\downarrow$ & Val PPL (final) $\downarrow$ & Mean entropy $\downarrow$ \\
\midrule
Additive (Bahdanau) & 26.30 & 0.539 & 7.3 & 7.2 & 1.942 \\
Multiplicative (Luong) & 25.24 & 0.514 & 8.3 & 8.2 & 0.000 \\
Scaled dot